In [1]:
import torch
from torch.autograd import Variable
import numpy as np
import pandas as pd
import sys
import random
import os
from IPython.display import clear_output
from matplotlib import pyplot as plt

#from scipy.stats import pearsonr
from scipy.stats import spearmanr
from evaluationUtils import r_square,get_cindex,pearson_r,pseudoAccuracy

import json

import seaborn as sns
sns.set()

In [2]:
device = torch.device('cuda')

# Load data

In [3]:
# Read data
cmap = pd.read_csv('cmap_HT29_A375.csv',index_col = 0)
#cmap_tf = pd.read_csv('../L1000_2021_11_23/cmap_compounds_tfs_repq1_tas03.tsv',
#                       sep='\t', low_memory=False, index_col=0)

gene_size = len(cmap.columns)
samples = cmap.index.values

In [4]:
# Create a train generators
def getSamples(N, batchSize):
    order = np.random.permutation(N)
    outList = []
    while len(order)>0:
        outList.append(order[0:batchSize])
        order = order[batchSize:]
    return outList

# Define fit model

In [5]:
class FITregressor(torch.nn.Module):
    
    def __init__(self, inputSize):
        
        super(FITregressor, self).__init__()
        
        alpha = torch.zeros(inputSize, requires_grad=True)
        beta = torch.ones(inputSize, requires_grad=True)
        self.alpha = torch.nn.Parameter(alpha)
        self.beta = torch.nn.Parameter(beta)
        
    def forward(self,x):
        y = self.alpha + x*self.beta
        return y
    
    def L1Regularization(self, L1=0.01):
        alphaReg = L1 * torch.sum(torch.abs(self.alpha),)
        betaReg = L1 * torch.sum(torch.abs(1-self.beta))
        return alphaReg+betaReg

In [6]:
def fitLoss(y_true,y_pred):
    return torch.mean(torch.sum(torch.square(y_true-y_pred),dim=1))

In [7]:
# fit = FITregressor(gene_size)
# fit = fit.to(device)

In [14]:
NUM_EPOCHS = 5000
bs = 64
#optimizer = torch.optim.Adam(fit.parameters(), lr=0.001, weight_decay=0)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.8)
l1 = 0.1
lr = 0.001

In [15]:
# Train with no batch (too few data)

trainPear = []
trainSpear = []
trainR2 =[]
trainMSE = []
trainAcc =[]

valPear = []
valSpear = []
valR2 =[]
valMSE = []
valAcc = []

for i in range(1):
    # do staff about loading paired data
    trainInfo_paired = pd.read_csv('10fold_validation_spit/train_paired_%s.csv'%i,index_col=0)
    valInfo_paired = pd.read_csv('10fold_validation_spit/val_paired_%s.csv'%i,index_col=0)
    N = len(trainInfo_paired)
    
    x1 = torch.tensor(cmap.loc[trainInfo_paired['sig_id.x'],:].values).to(device)
    #sig_id.x to translate from A375 to HT29
    x2 = torch.tensor(cmap.loc[trainInfo_paired['sig_id.y'],:].values).to(device)
    
    xval1 = torch.tensor(cmap.loc[valInfo_paired['sig_id.x'],:].values).to(device)
    #sig_id.x to translate from A375 to HT29
    xval2 = torch.tensor(cmap.loc[valInfo_paired['sig_id.y'],:].values).to(device)
    
    fit = FITregressor(gene_size)
    fit = fit.to(device)
    
    optimizer = torch.optim.Adam(fit.parameters(), lr=lr, weight_decay=0)
    
    for e in range(NUM_EPOCHS):
        #trainloader = getSamples(N, bs)
        fit.train()
        #for dataIndex in trainloader:
        optimizer.zero_grad()
        #x=
        y = fit(x1)
        loss = fitLoss(x2,y) #+ fit.L1Regularization(L1=l1)
        
        loss.backward()
        optimizer.step()
    
        pearson = pearson_r(y.detach(), x2.detach())
        r2 = r_square(y.detach().flatten(), x2.detach().flatten())
        mse = torch.mean(torch.mean((y.detach() - x2.detach())**2,dim=1))
        
        outString = 'Split {:.0f}: Epoch={:.0f}/{:.0f}'.format(i+1,e+1,NUM_EPOCHS)
        outString += ', r2={:.4f}'.format(r2.item())
        outString += ', pearson={:.4f}'.format(pearson.item())
        outString += ', MSE={:.4f}'.format(mse.item())
        outString += ', loss={:.4f}'.format(loss.item())
        print(outString)
        clear_output(wait=True)
    
    fit.eval()
    
    #x =
    y = fit(x1)
    
    pearson = pearson_r(y.detach(), x2.detach())
    r2 = r_square(y.detach().flatten(), x2.detach().flatten())
    mse = torch.mean(torch.mean((y.detach() - x2.detach())**2,dim=1))
    trainMSE.append(mse.item())
    trainPear.append(pearson.item())
    trainR2.append(r2.item())
    #rhos =[]
    #for jj in range(y.shape[0]):
    #    rho,p = spearmanr(x2[jj,:].detach().cpu().numpy(),y[jj,:].detach().cpu().numpy())
    #    rhos.append(rho)
    #print('Validation spearman: %s'%np.mean(rhos))
    #trainSpear.append(np.mean(rhos))
    #trainAcc = pseudoAccuracy(x2.detach().cpu(),y.detach().cpu(),eps=1e-4)
    
    #xval = 
    yval = fit(xval1)
    
    pearson = pearson_r(yval.detach(), xval2.detach())
    r2 = r_square(yval.detach().flatten(), xval2.detach().flatten())
    mse = torch.mean(torch.mean((yval.detach() - xval2.detach())**2,dim=1))
    
    valMSE.append(mse.item())
    valPear.append(pearson.item())
    valR2.append(r2.item())
    #rhos =[]
    #for jj in range(yval.shape[0]):
    #    rho,p = spearmanr(xval2[jj,:].detach().cpu().numpy(),yval[jj,:].detach().cpu().numpy())
    #    rhos.append(rho)
    #print('Validation spearman: %s'%np.mean(rhos))
    #valSpear.append(np.mean(rhos))
    #valAcc = pseudoAccuracy(xval2.detach().cpu(),yval.detach().cpu(),eps=1e-4)

Split 1: Epoch=5000/5000, r2=-1.7367, pearson=0.3453, MSE=0.8675, loss=8749.6379


In [16]:
print(valPear)
print(trainPear)

[0.40480477275813476]
[0.3453284472447295]


In [ ]:
# sns.displot(valPear)

In [ ]:
# print('Validation mean accuracy: %s'%np.mean(valAcc))

In [ ]:
# sns.displot(trainPear)

In [ ]:
# print('Training mean accuracy: %s'%np.mean(trainAcc))